<a href="https://colab.research.google.com/github/adrianmoses/text-search-nlp/blob/main/BertQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch faiss-gpu sentence-transformers

     |████████████████████████████████| 3.1 MB 11.5 MB/s 
     |████████████████████████████████| 89.7 MB 19 kB/s 
     |████████████████████████████████| 78 kB 9.7 MB/s 
     |████████████████████████████████| 59 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
     |████████████████████████████████| 3.3 MB 18.0 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
     |████████████████████████████████| 1.2 MB 33.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=c6cdbdff26897e5ed8ac607174db81aaccdd8394b296dfc13a1f96df5f962be0
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import faiss
import torch
from sentence_transformers import SentenceTransformer
import json

In [3]:
index = faiss.read_index('drive/MyDrive/cdc_search.index')

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
  
tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")

model = AutoModelForQuestionAnswering.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")

Downloading:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/717 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/224M [00:00<?, ?B/s]

In [5]:
embedder = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from transformers import pipeline
qa = pipeline("question-answering", model="ktrapeznikov/albert-xlarge-v2-squad-v2")

In [12]:
with open('drive/MyDrive/wiki_health_data.json') as f:
  data = json.load(f)

In [26]:
def search(query: str, k=1):
    encoded_query = embedder.encode([query])
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [data[_id] for _id in top_k[1][0]]
    answers = []
    for result in results:
        answer = qa(question=query, context=result['text'])
        if 'answer' in answer:
            answers.append((answer['answer'], answer['score']))
    return sorted(answers, key=lambda tup: tup[1], reverse=True)

In [27]:
search("How to prevent the spread of viral infections?")

[(' administration of vaccines', 0.05168561264872551)]